In [1]:
import os, openai
import dotenv
from langchain.tools import Tool
from dotenv import load_dotenv
from langchain.chains.base import Chain
from langchain.agents import Tool
from googleapiclient.discovery import build


dotenv.load_dotenv('../.env')
GOOGLE_API_KEY = os.getenv("GOOGLE-SEARCH-API-KEY")
SEARCH_ENGINE_ID = os.getenv("SEARCH-ENGINE-ID")

openai.api_key = os.getenv("CHATGPT-RECEIPT")

In [2]:
def search_with_google_api(query, num_results=5):
    api_key = GOOGLE_API_KEY  # Google API 키
    cse_id = SEARCH_ENGINE_ID   # Custom Search Engine ID
    try:
        service = build("customsearch", "v1", developerKey=api_key)
        results = service.cse().list(q=query, cx=cse_id, num=num_results).execute()
        items = results.get("items", [])
        return [
            {
                "title": item["title"], "snippet": item.get("snippet")  # 본문 내용 추가
            } 
            for item in items
        ]
    except Exception as e:
        return f'Google Search API Error: {e}'


class SearchChain(Chain):
    def __init__(self, tool):
        super().__init__()
        self._tool = tool

    @property
    def input_keys(self):
        return ["business_name"]

    @property
    def output_keys(self):
        return ["search_results"]

    def _call(self, inputs, **kwargs):
        business_name = inputs["business_name"]
        search_results = self._tool.func(business_name)
        return {"search_results": search_results}


# Tool로 검색 기능 정의
search_tool = Tool(
    name = "SearchBusinessCategory",
    func = lambda business_name: "\n".join(
        list([x['snippet'] for x in search_with_google_api(business_name)])
    ),
    description="상호명을 검색하여 관련 업종 정보를 제공합니다."
)

# Search chain
search_chain = SearchChain(search_tool)

In [3]:
# Search chain invoke 예제 코드

# 검색할 상호명
business_name = "스타벅스"

# Search chain 호출
inputs = {"business_name": business_name}
result = search_chain.invoke(inputs)

# 결과 출력
print(result["search_results"])


스타벅스 코리아 · COFFEE · 커피 · 스타벅스 원두 · MENU · 음료 · 콜드 브루 · STORE · 매장 찾기 · 퀵 검색 · RESPONSIBILITY · 사회공헌 캠페인 소개 · 지역 사회 참여 ...
Good at participating U.S. stores for a handcrafted menu-sized beverage with qualifying purchase ($8 max value). Qualifying purchase excludes alcohol, Starbucks ...
상품 · 백자 양각 매화문 세라믹 티팟 560ml. NEW · 모카 포트&원두 스쿱 세트 · 크림 컴프레소 · SS 그린 사이렌 케틀 550ml · 사이렌 고케틀.
Jan 1, 2025 ... 스타벅스라는 이름은 허먼 멜빌의 유명한 소설 모비 딕에 등장하는 포경선 피쿼드(Pequod)호의 일등항해사 스타벅(Starbuck)에서 's'를 붙여서 탄생하게 ...
사이렌 오더. ... 서비스 이용을 위해 권한 허용이 필요합니다. 허용하지 않으셔도 앱 이용은 가능하나 일부 서비스 이용에 제한이 있을 수 있습니다. ... * 스타벅스 앱의 ...
